In [4]:
import cv2
import numpy as np
import mediapipe as mp
import time
import os
import sys
import random

try:
    import winsound
    SOUND_AVAILABLE = True
except ImportError:
    SOUND_AVAILABLE = False
    print("Winsound not available - sound feedback disabled")

# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    static_image_mode=False,
    max_num_faces=1,
    min_detection_confidence=0.5, 
    min_tracking_confidence=0.5
)

# Eye landmark indices (right and left eye)
LEFT_EYE = [33, 160, 158, 133, 153, 144]
RIGHT_EYE = [362, 385, 387, 263, 373, 380]

# Font for displaying text
font = cv2.FONT_HERSHEY_SIMPLEX

# Dark theme color palette - using BGR format for OpenCV
COLORS = {
    'background': (20, 20, 20),        # Dark background (near black)
    'panel': (40, 40, 40),             # Dark gray panels
    'text': (255, 255, 255),           # White text
    'accent': (0, 117, 255),           # Blue accent (player X)
    'success': (76, 187, 23),          # Green (win highlight)
    'warning': (0, 149, 255),          # Orange/amber (player O)
    'cursor': (0, 117, 255),           # Blue cursor
    'grid': (70, 70, 70),              # Grid lines
    'cell': (50, 50, 50),              # Cell background
    'cell_hover': (60, 60, 60),        # Cell hover state
    'header': (15, 15, 15)             # Header bar even darker
}

# Initialize game interface
game_interface = np.zeros((700, 1000, 3), np.uint8)

# Variables for tracking
blink_counter = 0
blink_threshold = 0.19  # EAR threshold for blink detection
blink_frames_required = 6  # Frames required to confirm a blink
blink_cooldown = 0  # Cooldown counter to prevent accidental double blinks
current_mode = "main_menu"  # Can be "main_menu", "game", or "game_over"
exit_requested = False  # Flag to track if exit was requested

# Cursor variables
cursor_pos = [500, 350]  # Initial cursor position
cursor_smooth_factor = 0.3  # Smoothing factor for cursor movement (0-1)
gaze_element = None  # Current element being gazed at
gaze_index = None  # Index of the current element

# Game variables
board = [None] * 9  # Tic tac toe board (3x3 grid flattened to 1D)
current_player = "X"  # X starts the game
game_winner = None
ai_enabled = True  # Enable/disable AI opponent
ai_difficulty = "Medium"  # AI difficulty levels: "Easy", "Medium", "Hard"
game_stats = {"X_wins": 0, "O_wins": 0, "Draws": 0}

# Function to calculate Eye Aspect Ratio (EAR)
def calculate_ear(eye_landmarks):
    vertical_1 = np.linalg.norm(np.array(eye_landmarks[1]) - np.array(eye_landmarks[5]))
    vertical_2 = np.linalg.norm(np.array(eye_landmarks[2]) - np.array(eye_landmarks[4]))
    horizontal = np.linalg.norm(np.array(eye_landmarks[0]) - np.array(eye_landmarks[3]))
    ear = (vertical_1 + vertical_2) / (2.0 * horizontal)
    return ear

# Function to smoothly update cursor position based on gaze
def update_cursor_position(current_pos, target_pos, smoothing_factor):
    # Apply smoothing to reduce jitter
    new_x = int(current_pos[0] + (target_pos[0] - current_pos[0]) * smoothing_factor)
    new_y = int(current_pos[1] + (target_pos[1] - current_pos[1]) * smoothing_factor)
    return [new_x, new_y]

# Function to check if point is inside rectangle
def is_point_in_rect(point, rect_start, rect_end):
    x, y = point
    x1, y1 = rect_start
    x2, y2 = rect_end
    return x1 <= x <= x2 and y1 <= y <= y2

# Function to create a simple rounded rectangle
def draw_rounded_rectangle(img, top_left, bottom_right, color, radius=6, thickness=-1):
    x1, y1 = top_left
    x2, y2 = bottom_right
    
    # Draw main rectangle
    cv2.rectangle(img, (x1+radius, y1), (x2-radius, y2), color, thickness)
    cv2.rectangle(img, (x1, y1+radius), (x2, y2-radius), color, thickness)
    
    # Draw four corners
    cv2.ellipse(img, (x1+radius, y1+radius), (radius, radius), 180, 0, 90, color, thickness)
    cv2.ellipse(img, (x2-radius, y1+radius), (radius, radius), 270, 0, 90, color, thickness)
    cv2.ellipse(img, (x1+radius, y2-radius), (radius, radius), 90, 0, 90, color, thickness)
    cv2.ellipse(img, (x2-radius, y2-radius), (radius, radius), 0, 0, 90, color, thickness)

# Function to draw cursor
def draw_cursor(interface, x, y):
    # Outer glow effect
    cv2.circle(interface, (x, y), 8, COLORS['cursor'], -1, cv2.LINE_AA)
    # Inner brighter circle
    cv2.circle(interface, (x, y), 3, (255, 255, 255), -1, cv2.LINE_AA)

# Function to check for a win or draw
def check_game_state():
    # Check rows
    for i in range(0, 9, 3):
        if board[i] is not None and board[i] == board[i+1] == board[i+2]:
            return board[i]  # Return the winner (X or O)
    
    # Check columns
    for i in range(3):
        if board[i] is not None and board[i] == board[i+3] == board[i+6]:
            return board[i]
    
    # Check diagonals
    if board[0] is not None and board[0] == board[4] == board[8]:
        return board[0]
    if board[2] is not None and board[2] == board[4] == board[6]:
        return board[2]
    
    # Check for draw (all cells filled)
    if all(cell is not None for cell in board):
        return "Draw"
    
    # Game is still ongoing
    return None

# AI move function with difficulty levels
def ai_make_move():
    global board, current_player
    
    # Easy: Random move
    if ai_difficulty == "Easy":
        empty_cells = [i for i, cell in enumerate(board) if cell is None]
        if empty_cells:
            return random.choice(empty_cells)
        return None
    
    # Medium and Hard: More strategic
    elif ai_difficulty in ["Medium", "Hard"]:
        # First check if AI can win in the next move
        for i in range(9):
            if board[i] is None:
                board[i] = "O"  # Try placing O
                if check_game_state() == "O":  # If this leads to a win
                    board[i] = None  # Reset
                    return i
                board[i] = None  # Reset
        
        # Then check if player can win in the next move (and block)
        for i in range(9):
            if board[i] is None:
                board[i] = "X"  # Try placing X
                if check_game_state() == "X":  # If this leads to player winning
                    board[i] = None  # Reset
                    return i
                board[i] = None  # Reset
        
        # Hard: Add some more strategy
        if ai_difficulty == "Hard":
            # Take center if available
            if board[4] is None:
                return 4
            
            # Take corners
            corners = [0, 2, 6, 8]
            empty_corners = [i for i in corners if board[i] is None]
            if empty_corners:
                return random.choice(empty_corners)
        
        # Otherwise, make a random move
        empty_cells = [i for i, cell in enumerate(board) if cell is None]
        if empty_cells:
            return random.choice(empty_cells)
    
    return None  # No move possible

# Draw the main menu
def draw_menu():
    global gaze_element, gaze_index, game_interface
    
    # Clear the interface with dark background
    game_interface[:] = COLORS['background']
    
    # Header bar 
    cv2.rectangle(game_interface, (0, 0), (1000, 60), COLORS['header'], -1)
    
    # Title
    cv2.putText(game_interface, "Eye-Controlled Tic Tac Toe", (40, 40), font, 1.2, COLORS['text'], 2)
    
    # Exit button
    exit_btn_x, exit_btn_y = 850, 40
    
    # Check if cursor is over exit button
    if is_point_in_rect(cursor_pos, (exit_btn_x - 40, exit_btn_y - 20), (exit_btn_x + 70, exit_btn_y + 10)):
        gaze_element = "exit"
        cv2.putText(game_interface, "Exit", (exit_btn_x, exit_btn_y), font, 0.7, COLORS['warning'], 1)
    else:
        cv2.putText(game_interface, "Exit", (exit_btn_x, exit_btn_y), font, 0.7, COLORS['text'], 1)
    
    # Reset gaze detection for other elements (keep exit if active)
    if gaze_element != "exit":
        gaze_element = None
        gaze_index = None
    
    # Game stats panel
    panel_x, panel_y = 40, 100
    panel_w, panel_h = 200, 200
    cv2.rectangle(game_interface, (panel_x, panel_y), 
                 (panel_x + panel_w, panel_y + panel_h), 
                 COLORS['panel'], -1)
    cv2.rectangle(game_interface, (panel_x, panel_y), 
                 (panel_x + panel_w, panel_y + panel_h), 
                 (60, 60, 60), 1)  # subtle border
    
    # Stats title
    cv2.putText(game_interface, "Game Statistics:", (panel_x + 15, panel_y + 35), 
               font, 0.7, COLORS['accent'], 1)
    
    # Show stats
    cv2.putText(game_interface, f"X Wins: {game_stats['X_wins']}", (panel_x + 20, panel_y + 80), 
               font, 0.7, COLORS['text'], 1)
    cv2.putText(game_interface, f"O Wins: {game_stats['O_wins']}", (panel_x + 20, panel_y + 120), 
               font, 0.7, COLORS['text'], 1)
    cv2.putText(game_interface, f"Draws: {game_stats['Draws']}", (panel_x + 20, panel_y + 160), 
               font, 0.7, COLORS['text'], 1)
    
    # Blink progress bar
    if blink_counter > 0:
        progress_bar_x = 780
        progress_bar_y = 80
        progress_bar_w = 200
        progress_bar_h = 10
        
        # Draw background track
        cv2.rectangle(game_interface, 
                     (progress_bar_x, progress_bar_y), 
                     (progress_bar_x + progress_bar_w, progress_bar_y + progress_bar_h), 
                     (40, 40, 40), -1, cv2.LINE_AA)
        
        # Draw progress
        progress = int((blink_counter / blink_frames_required) * progress_bar_w)
        cv2.rectangle(game_interface, 
                     (progress_bar_x, progress_bar_y), 
                     (progress_bar_x + progress, progress_bar_y + progress_bar_h), 
                     COLORS['accent'], -1, cv2.LINE_AA)
        
        # Add label
        cv2.putText(game_interface, "Blink", (progress_bar_x - 40, progress_bar_y + 9), 
                   font, 0.5, COLORS['text'], 1)
    
    # AI options panel
    ai_panel_x, ai_panel_y = 40, 330
    ai_panel_w, ai_panel_h = 200, 160
    cv2.rectangle(game_interface, (ai_panel_x, ai_panel_y), 
                 (ai_panel_x + ai_panel_w, ai_panel_y + ai_panel_h), 
                 COLORS['panel'], -1)
    cv2.rectangle(game_interface, (ai_panel_x, ai_panel_y), 
                 (ai_panel_x + ai_panel_w, ai_panel_y + ai_panel_h), 
                 (60, 60, 60), 1)
    
    # AI settings title
    cv2.putText(game_interface, "AI Settings:", (ai_panel_x + 15, ai_panel_y + 35), 
               font, 0.7, COLORS['accent'], 1)
    
    # AI toggle button
    ai_toggle_x, ai_toggle_y = ai_panel_x + 20, ai_panel_y + 80
    ai_toggle_w, ai_toggle_h = 160, 40
    
    # Check if cursor is over AI toggle
    if is_point_in_rect(cursor_pos, (ai_toggle_x, ai_toggle_y), 
                      (ai_toggle_x + ai_toggle_w, ai_toggle_y + ai_toggle_h)):
        gaze_element = "ai_toggle"
        draw_rounded_rectangle(game_interface, (ai_toggle_x, ai_toggle_y), 
                    (ai_toggle_x + ai_toggle_w, ai_toggle_y + ai_toggle_h), 
                    COLORS['accent'], 6)
        font_color = (255, 255, 255)
    else:
        draw_rounded_rectangle(game_interface, (ai_toggle_x, ai_toggle_y), 
                    (ai_toggle_x + ai_toggle_w, ai_toggle_y + ai_toggle_h), 
                    COLORS['panel'], 6)
        cv2.rectangle(game_interface, (ai_toggle_x, ai_toggle_y), 
                     (ai_toggle_x + ai_toggle_w, ai_toggle_y + ai_toggle_h), 
                     (60, 60, 60), 1)
        font_color = COLORS['text']
    
    # AI toggle text
    ai_text = f"AI: {'ON' if ai_enabled else 'OFF'}"
    cv2.putText(game_interface, ai_text, (ai_toggle_x + 40, ai_toggle_y + 28), 
               font, 0.8, font_color, 2)
    
    # AI difficulty button (only shown if AI is enabled)
    if ai_enabled:
        difficulty_x, difficulty_y = ai_panel_x + 20, ai_panel_y + 130
        difficulty_w, difficulty_h = 160, 40
        
        # Check if cursor is over difficulty button
        if is_point_in_rect(cursor_pos, (difficulty_x, difficulty_y), 
                          (difficulty_x + difficulty_w, difficulty_y + difficulty_h)):
            gaze_element = "difficulty"
            draw_rounded_rectangle(game_interface, (difficulty_x, difficulty_y), 
                        (difficulty_x + difficulty_w, difficulty_y + difficulty_h), 
                        COLORS['accent'], 6)
            font_color = (255, 255, 255)
        else:
            draw_rounded_rectangle(game_interface, (difficulty_x, difficulty_y), 
                        (difficulty_x + difficulty_w, difficulty_y + difficulty_h), 
                        COLORS['panel'], 6)
            cv2.rectangle(game_interface, (difficulty_x, difficulty_y), 
                         (difficulty_x + difficulty_w, difficulty_y + difficulty_h), 
                         (60, 60, 60), 1)
            font_color = COLORS['text']
        
        # Difficulty text
        diff_text = f"Difficulty: {ai_difficulty}"
        cv2.putText(game_interface, diff_text, (difficulty_x + 15, difficulty_y + 28), 
                   font, 0.6, font_color, 2)
    
    # Draw play button (centered and large)
    play_btn_x, play_btn_y = 400, 250
    play_btn_w, play_btn_h = 220, 80
    
    # Check if cursor is over play button
    if is_point_in_rect(cursor_pos, (play_btn_x, play_btn_y), 
                      (play_btn_x + play_btn_w, play_btn_y + play_btn_h)):
        gaze_element = "play"
        draw_rounded_rectangle(game_interface, (play_btn_x, play_btn_y), 
                    (play_btn_x + play_btn_w, play_btn_y + play_btn_h), 
                    COLORS['success'], 8)
        font_color = (255, 255, 255)
    else:
        draw_rounded_rectangle(game_interface, (play_btn_x, play_btn_y), 
                    (play_btn_x + play_btn_w, play_btn_y + play_btn_h), 
                    COLORS['panel'], 8)
        cv2.rectangle(game_interface, (play_btn_x, play_btn_y), 
                     (play_btn_x + play_btn_w, play_btn_y + play_btn_h), 
                     (60, 60, 60), 1)
        font_color = COLORS['text']
    
    # Play text
    cv2.putText(game_interface, "PLAY GAME", (play_btn_x + 32, play_btn_y + 50), 
               font, 1.1, font_color, 2)
    
    # Draw cursor
    draw_cursor(game_interface, cursor_pos[0], cursor_pos[1])

# Draw the game board
def draw_game():
    global gaze_element, gaze_index, game_interface, current_player
    
    # Clear the interface with dark background
    game_interface[:] = COLORS['background']
    
    # Header bar
    cv2.rectangle(game_interface, (0, 0), (1000, 60), COLORS['header'], -1)
    
    # Title
    cv2.putText(game_interface, "Tic Tac Toe", (40, 40), font, 1.2, COLORS['text'], 2)
    
    # Current player indicator
    player_color = COLORS['accent'] if current_player == "X" else COLORS['warning']
    cv2.putText(game_interface, f"Current Player: {current_player}", (350, 40), 
               font, 0.8, player_color, 2)
    
    # Exit button
    exit_btn_x, exit_btn_y = 850, 40
    
    # Check if cursor is over exit button
    if is_point_in_rect(cursor_pos, (exit_btn_x - 40, exit_btn_y - 20), (exit_btn_x + 70, exit_btn_y + 10)):
        gaze_element = "exit"
        cv2.putText(game_interface, "Exit", (exit_btn_x, exit_btn_y), font, 0.7, COLORS['warning'], 1)
    else:
        cv2.putText(game_interface, "Exit", (exit_btn_x, exit_btn_y), font, 0.7, COLORS['text'], 1)
    
    # Reset gaze detection for other elements (keep exit if active)
    if gaze_element != "exit":
        gaze_element = None
        gaze_index = None
    
    # Blink progress bar
    if blink_counter > 0:
        progress_bar_x = 780
        progress_bar_y = 80
        progress_bar_w = 200
        progress_bar_h = 10
        
        # Draw background track
        cv2.rectangle(game_interface, 
                     (progress_bar_x, progress_bar_y), 
                     (progress_bar_x + progress_bar_w, progress_bar_y + progress_bar_h), 
                     (40, 40, 40), -1, cv2.LINE_AA)
        
        # Draw progress
        progress = int((blink_counter / blink_frames_required) * progress_bar_w)
        cv2.rectangle(game_interface, 
                     (progress_bar_x, progress_bar_y), 
                     (progress_bar_x + progress, progress_bar_y + progress_bar_h), 
                     player_color, -1, cv2.LINE_AA)
        
        # Add label
        cv2.putText(game_interface, "Blink", (progress_bar_x - 40, progress_bar_y + 9), 
                   font, 0.5, COLORS['text'], 1)
    
    # Draw the tic-tac-toe grid (centered on screen)
    grid_size = 450  # Total grid size
    cell_size = grid_size // 3
    grid_x = (1000 - grid_size) // 2
    grid_y = 130
    
    # Draw cells
    for row in range(3):
        for col in range(3):
            # Calculate cell position
            cell_index = row * 3 + col
            cell_x = grid_x + col * cell_size
            cell_y = grid_y + row * cell_size
            
            # Check if cursor is over this cell
            if is_point_in_rect(cursor_pos, (cell_x, cell_y), 
                              (cell_x + cell_size, cell_y + cell_size)):
                # Cell is being gazed at and is empty
                if board[cell_index] is None:
                    gaze_element = "cell"
                    gaze_index = cell_index
                    cell_color = COLORS['cell_hover']
                else:
                    cell_color = COLORS['cell']
            else:
                cell_color = COLORS['cell']
            
            # Draw cell background
            cv2.rectangle(game_interface, (cell_x, cell_y), 
                         (cell_x + cell_size, cell_y + cell_size), 
                         cell_color, -1)
            
            # Draw cell border
            cv2.rectangle(game_interface, (cell_x, cell_y), 
                         (cell_x + cell_size, cell_y + cell_size), 
                         COLORS['grid'], 2)
            
            # Draw X or O if cell is marked
            if board[cell_index] is not None:
                symbol = board[cell_index]
                symbol_color = COLORS['accent'] if symbol == "X" else COLORS['warning']
                
                # Calculate text position for centering
                text_size = cv2.getTextSize(symbol, font, 3, 3)[0]
                text_x = cell_x + (cell_size - text_size[0]) // 2
                text_y = cell_y + (cell_size + text_size[1]) // 2
                
                cv2.putText(game_interface, symbol, (text_x, text_y), 
                           font, 3, symbol_color, 3)
    
    # Draw "Menu" button
    menu_btn_x, menu_btn_y = 400, 620
    menu_btn_w, menu_btn_h = 200, 50
    
    # Check if cursor is over menu button
    if is_point_in_rect(cursor_pos, (menu_btn_x, menu_btn_y), 
                      (menu_btn_x + menu_btn_w, menu_btn_y + menu_btn_h)):
        gaze_element = "menu"
        draw_rounded_rectangle(game_interface, (menu_btn_x, menu_btn_y), 
                    (menu_btn_x + menu_btn_w, menu_btn_y + menu_btn_h), 
                    COLORS['accent'], 6)
        font_color = (255, 255, 255)
    else:
        draw_rounded_rectangle(game_interface, (menu_btn_x, menu_btn_y), 
                    (menu_btn_x + menu_btn_w, menu_btn_y + menu_btn_h), 
                    COLORS['panel'], 6)
        cv2.rectangle(game_interface, (menu_btn_x, menu_btn_y), 
                     (menu_btn_x + menu_btn_w, menu_btn_y + menu_btn_h), 
                     (60, 60, 60), 1)
        font_color = COLORS['text']
    
    # Menu text
    cv2.putText(game_interface, "MAIN MENU", (menu_btn_x + 40, menu_btn_y + 35), 
               font, 0.8, font_color, 2)
    
    # Draw cursor
    draw_cursor(game_interface, cursor_pos[0], cursor_pos[1])

# Draw the game over screen
def draw_game_over():
    global gaze_element, gaze_index, game_interface, game_winner
    
    # Draw the game board in the background
    draw_game()
    
    # Add semi-transparent overlay
    overlay = game_interface.copy()
    cv2.rectangle(overlay, (0, 0), (1000, 700), (0, 0, 0), -1)
    cv2.addWeighted(overlay, 0.7, game_interface, 0.3, 0, game_interface)
    
    # Reset gaze detection
    gaze_element = None
    gaze_index = None
    
    # Game over message
    if game_winner == "Draw":
        result_text = "Game Over - It's a Draw!"
        text_color = COLORS['text']
    else:
        result_text = f"Game Over - {game_winner} Wins!"
        text_color = COLORS['accent'] if game_winner == "X" else COLORS['warning']
    
    # Display result (centered)
    text_size = cv2.getTextSize(result_text, font, 1.5, 3)[0]
    text_x = (1000 - text_size[0]) // 2
    cv2.putText(game_interface, result_text, (text_x, 300), 
               font, 1.5, text_color, 3)
    
    # Play again button
    play_again_x, play_again_y = 350, 380
    play_again_w, play_again_h = 300, 60
    
    # Check if cursor is over play again button
    if is_point_in_rect(cursor_pos, (play_again_x, play_again_y), 
                      (play_again_x + play_again_w, play_again_y + play_again_h)):
        gaze_element = "play_again"
        draw_rounded_rectangle(game_interface, (play_again_x, play_again_y), 
                    (play_again_x + play_again_w, play_again_y + play_again_h), 
                    COLORS['success'], 8)
        font_color = (255, 255, 255)
    else:
        draw_rounded_rectangle(game_interface, (play_again_x, play_again_y), 
                    (play_again_x + play_again_w, play_again_y + play_again_h), 
                    COLORS['panel'], 8)
        cv2.rectangle(game_interface, (play_again_x, play_again_y), 
                     (play_again_x + play_again_w, play_again_y + play_again_h), 
                     (60, 60, 60), 1)
        font_color = COLORS['text']
    
    # Play again text
    cv2.putText(game_interface, "PLAY AGAIN", (play_again_x + 70, play_again_y + 40), 
               font, 1, font_color, 2)
    
    # Main menu button
    menu_btn_x, menu_btn_y = 350, 470
    menu_btn_w, menu_btn_h = 300, 60
    
    # Check if cursor is over menu button
    if is_point_in_rect(cursor_pos, (menu_btn_x, menu_btn_y), 
                      (menu_btn_x + menu_btn_w, menu_btn_y + menu_btn_h)):
        gaze_element = "game_over_menu"
        draw_rounded_rectangle(game_interface, (menu_btn_x, menu_btn_y), 
                    (menu_btn_x + menu_btn_w, menu_btn_y + menu_btn_h), 
                    COLORS['accent'], 8)
        font_color = (255, 255, 255)
    else:
        draw_rounded_rectangle(game_interface, (menu_btn_x, menu_btn_y), 
                    (menu_btn_x + menu_btn_w, menu_btn_y + menu_btn_h), 
                    COLORS['panel'], 8)
        cv2.rectangle(game_interface, (menu_btn_x, menu_btn_y), 
                     (menu_btn_x + menu_btn_w, menu_btn_y + menu_btn_h), 
                     (60, 60, 60), 1)
        font_color = COLORS['text']
    
    # Menu text
    cv2.putText(game_interface, "MAIN MENU", (menu_btn_x + 80, menu_btn_y + 40), 
               font, 1, font_color, 2)
    
    # Draw cursor
    draw_cursor(game_interface, cursor_pos[0], cursor_pos[1])

# Function to update game state
def update_game_state(cell_index):
    global board, current_player, game_winner, game_stats
    
    # If cell is already occupied, do nothing
    if board[cell_index] is not None:
        return
    
    # Place the current player's mark
    board[cell_index] = current_player
    
    # Check for win or draw
    result = check_game_state()
    
    if result is not None:
        # Game is over
        game_winner = result
        
        # Update stats
        if result == "Draw":
            game_stats["Draws"] += 1
        elif result == "X":
            game_stats["X_wins"] += 1
        else:  # O wins
            game_stats["O_wins"] += 1
            
        # Change to game over screen
        return "game_over"
    
    # Switch player
    current_player = "O" if current_player == "X" else "X"
    
    # If AI is enabled and it's O's turn, make AI move
    if ai_enabled and current_player == "O":
        # Add small delay to make AI move seem more natural
        cv2.waitKey(500)
        
        # Get AI move
        ai_move = ai_make_move()
        
        if ai_move is not None:
            # Place O at the selected position
            board[ai_move] = "O"
            
            # Check for win or draw again
            result = check_game_state()
            
            if result is not None:
                # Game is over
                game_winner = result
                
                # Update stats
                if result == "Draw":
                    game_stats["Draws"] += 1
                elif result == "O":
                    game_stats["O_wins"] += 1
                    
                # Change to game over screen
                return "game_over"
            
            # Switch back to player X
            current_player = "X"
    
    return "continue"

# Function to reset game
def reset_game():
    global board, current_player, game_winner
    
    # Clear the board
    board = [None] * 9
    
    # X always starts
    current_player = "X"
    
    # Reset winner
    game_winner = None

# Main program entry point
if __name__ == "__main__":
    try:
        # Initialize camera
        cap = cv2.VideoCapture(0)
        
        # Check if camera is opened successfully
        if not cap.isOpened():
            print("Error: Could not open camera.")
            exit()
        
        # Create and initialize windows
        cv2.namedWindow("Camera", cv2.WINDOW_NORMAL)
        cv2.namedWindow("Tic Tac Toe", cv2.WINDOW_NORMAL)
        
        # Resize windows to reasonable dimensions
        cv2.resizeWindow("Camera", 640, 480)
        cv2.resizeWindow("Tic Tac Toe", 1000, 700)
        
        # Pre-initialize the interface
        game_interface = np.zeros((700, 1000, 3), np.uint8)
        game_interface[:] = COLORS['background']
        
        # Initialize cursor position
        cursor_pos = [500, 350]
        
        # First draw of the menu
        draw_menu()
        cv2.imshow("Tic Tac Toe", game_interface)
        
        print("Eye-Controlled Tic Tac Toe starting...")
        print("Press ESC to exit or use the Exit button in the interface")
        
        while True:
            if exit_requested:
                print("Exit requested. Closing application...")
                break
                
            ret, frame = cap.read()
            if not ret:
                print("Error: Failed to capture image")
                break
                
            # Flip for mirror effect
            frame = cv2.flip(frame, 1)
            
            # Convert to RGB for MediaPipe
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = face_mesh.process(rgb_frame)
            
            if results.multi_face_landmarks:
                for face_landmarks in results.multi_face_landmarks:
                    # Get eye landmarks
                    left_eye = [(face_landmarks.landmark[i].x * frame.shape[1], face_landmarks.landmark[i].y * frame.shape[0]) 
                                for i in LEFT_EYE]
                    right_eye = [(face_landmarks.landmark[i].x * frame.shape[1], face_landmarks.landmark[i].y * frame.shape[0]) 
                                 for i in RIGHT_EYE]

                    # Calculate EAR
                    left_ear = calculate_ear(left_eye)
                    right_ear = calculate_ear(right_eye)
                    avg_ear = (left_ear + right_ear) / 2

                    # Draw eye landmarks
                    for point in left_eye + right_eye:
                        cv2.circle(frame, (int(point[0]), int(point[1])), 2, COLORS['accent'], -1, cv2.LINE_AA)
                        
                    # Simple gaze estimation using eye midpoints
                    left_mid_x = sum(p[0] for p in left_eye) / len(left_eye)
                    left_mid_y = sum(p[1] for p in left_eye) / len(left_eye)
                    right_mid_x = sum(p[0] for p in right_eye) / len(right_eye)
                    right_mid_y = sum(p[1] for p in right_eye) / len(right_eye)
                    
                    eye_mid_x = (left_mid_x + right_mid_x) / 2
                    eye_mid_y = (left_mid_y + right_mid_y) / 2
                    
                    # Scale from camera frame to interface dimensions
                    # Add sensitivity multipliers
                    sensitivity_x = 4.5  # Horizontal sensitivity
                    sensitivity_y = 4.5  # Vertical sensitivity

                    # Center point of the frame (neutral position)
                    center_x = frame.shape[1] / 2
                    center_y = frame.shape[0] / 2

                    # Calculate offset from center and apply sensitivity
                    offset_x = (eye_mid_x - center_x) * sensitivity_x
                    offset_y = (eye_mid_y - center_y) * sensitivity_y

                    # Apply offset to center of interface
                    target_x = int(game_interface.shape[1] / 2 + offset_x)
                    target_y = int(game_interface.shape[0] / 2 + offset_y)

                    # Ensure target stays within bounds
                    target_x = max(0, min(game_interface.shape[1] - 1, target_x))
                    target_y = max(0, min(game_interface.shape[0] - 1, target_y))
                    
                    # Update cursor position with smoothing
                    cursor_pos = update_cursor_position(cursor_pos, [target_x, target_y], cursor_smooth_factor)

                    # Blink detection logic with improved filtering and cooldown
                    if blink_cooldown > 0:
                        # Decrement cooldown counter
                        blink_cooldown -= 1
                        blink_counter = 0  # Reset counter during cooldown
                    elif avg_ear < blink_threshold:
                        # Only count consecutive frames below threshold
                        blink_counter += 1
                        # Visual feedback for blink detection
                        if blink_counter > 2:  # Only show visual feedback after a few frames
                            cv2.putText(frame, "BLINK", (frame.shape[1] - 120, 40), font, 1, COLORS['accent'], 2, cv2.LINE_AA)
                    else:
                        if blink_counter >= blink_frames_required:
                            # Selection confirmed with a blink
                            # Set cooldown to prevent immediate new blink detection
                            blink_cooldown = 15
                            
                            # Play sound for feedback if available
                            if SOUND_AVAILABLE:
                                try:
                                    winsound.PlaySound("SystemExclamation", winsound.SND_ALIAS)
                                except:
                                    pass
                            
                            # Handle selection based on current mode and cursor position
                            # Check for exit first (available in all modes)
                            if gaze_element == "exit":
                                # Exit button selected
                                exit_requested = True
                                print("Exit requested via blink. Closing application...")
                                break
                            
                            # Handle main menu selections
                            elif current_mode == "main_menu":
                                if gaze_element == "play":
                                    # Start a new game
                                    reset_game()
                                    current_mode = "game"
                                    print("Starting new game.")
                                
                                elif gaze_element == "ai_toggle":
                                    # Toggle AI on/off
                                    ai_enabled = not ai_enabled
                                    print(f"AI opponent {'enabled' if ai_enabled else 'disabled'}")
                                
                                elif gaze_element == "difficulty" and ai_enabled:
                                    # Cycle through difficulty levels
                                    if ai_difficulty == "Easy":
                                        ai_difficulty = "Medium"
                                    elif ai_difficulty == "Medium":
                                        ai_difficulty = "Hard"
                                    else:
                                        ai_difficulty = "Easy"
                                    print(f"AI difficulty set to {ai_difficulty}")
                            
                            # Handle game selections
                            elif current_mode == "game":
                                if gaze_element == "cell" and gaze_index is not None:
                                    # Make a move in the selected cell
                                    result = update_game_state(gaze_index)
                                    if result == "game_over":
                                        current_mode = "game_over"
                                        print(f"Game over: {game_winner}")
                                
                                elif gaze_element == "menu":
                                    # Return to main menu
                                    current_mode = "main_menu"
                                    print("Returning to main menu.")
                            
                            # Handle game over selections
                            elif current_mode == "game_over":
                                if gaze_element == "play_again":
                                    # Start a new game
                                    reset_game()
                                    current_mode = "game"
                                    print("Starting new game.")
                                
                                elif gaze_element == "game_over_menu":
                                    # Return to main menu
                                    current_mode = "main_menu"
                                    print("Returning to main menu.")
                        
                        blink_counter = 0  # Reset blink counter
            
            # Draw the appropriate interface
            if current_mode == "main_menu":
                draw_menu()
            elif current_mode == "game":
                draw_game()
            elif current_mode == "game_over":
                draw_game_over()

            # Add subtle border to camera frame
            cv2.rectangle(frame, (0, 0), (frame.shape[1]-1, frame.shape[0]-1), (40, 40, 40), 1)

            # Show windows
            cv2.imshow("Camera", frame)
            cv2.imshow("Tic Tac Toe", game_interface)
            
            key = cv2.waitKey(1)
            if key == 27:  # Press ESC to exit
                break
            
            # Additional keyboard shortcuts for testing
            elif key == ord('m'):  # Press 'm' to return to main menu
                current_mode = "main_menu"
                
            elif key == ord('p'):  # Press 'p' to start playing
                if current_mode == "main_menu":
                    reset_game()
                    current_mode = "game"
                
            elif key == ord('r'):  # Press 'r' to reset the game
                reset_game()
                if current_mode == "game_over":
                    current_mode = "game"
                
            elif key == ord('a'):  # Press 'a' to toggle AI
                ai_enabled = not ai_enabled
                
            elif key == ord('d'):  # Press 'd' to cycle through difficulty
                if ai_difficulty == "Easy":
                    ai_difficulty = "Medium"
                elif ai_difficulty == "Medium":
                    ai_difficulty = "Hard"
                else:
                    ai_difficulty = "Easy"
                
            elif key == ord('q') or key == ord('x'):  # Additional exit keys
                exit_requested = True
                break
        
        # Clean up
        print("Exiting Eye-Controlled Tic Tac Toe...")
        cap.release()
        cv2.destroyAllWindows()
        
    except Exception as e:
        print(f"An error occurred: {e}")
        
    finally:
        # Final cleanup to ensure resources are freed
        print("Cleaning up resources...")
        if 'cap' in locals() and cap is not None:
            cap.release()
        
        cv2.destroyAllWindows()
        cv2.waitKey(100)  # Give time for windows to close
        cv2.destroyAllWindows()  # Call again to ensure all windows close

Eye-Controlled Tic Tac Toe starting...
Press ESC to exit or use the Exit button in the interface
Starting new game.
Game over: O
Returning to main menu.
AI difficulty set to Hard
Starting new game.
Game over: O
Cleaning up resources...


KeyboardInterrupt: 